In [ ]:
!pip install -r /content/drive/MyDrive/003/requirements.txt

In [ ]:
!pip install --upgrade accelerate

In [ ]:
import pandas as pd
import numpy as np
import datetime
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import re,os
import unicodedata
import matplotlib.pyplot as plt
import itertools
import sklearn.metrics as metrics
from sklearn import metrics, feature_selection
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, brier_score_loss,accuracy_score,classification_report,accuracy_score,confusion_matrix,recall_score,precision_score,roc_curve
from transformers import Trainer, TrainingArguments, EvalPrediction,DataCollatorWithPadding, Trainer, TrainingArguments, AutoModelForSequenceClassification
from transformers import BertModel, BertConfig, BertTokenizer, DebertaConfig, DebertaModel, DebertaTokenizer,DebertaV2Model, DebertaV2Config,DebertaV2Tokenizer,AutoTokenizer,AutoModel,AutoConfig
import optuna.visualization as optuna_visualization
import plotly
import numpy as np
import json

from transformers.modeling_outputs import SequenceClassifierOutput
from bs4 import BeautifulSoup

import optuna
from optuna.visualization import plot_optimization_history


In [ ]:
device=torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

In [ ]:
# CON DEBERTA
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
config = DebertaConfig.from_pretrained("microsoft/deberta-base", output_hidden_states=True, output_attentions=True)
MODEL = DebertaModel.from_pretrained("microsoft/deberta-base", config=config)
MODEL_TYPE = 'deberta'

# CON MDEBERTA

# tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
# config = AutoConfig.from_pretrained("microsoft/mdeberta-v3-base",output_hidden_states=True, output_attentions=True)
# MODEL = AutoModel.from_pretrained("microsoft/mdeberta-v3-base", config=config)
# MODEL_TYPE = 'mdeberta'

In [ ]:
now = datetime.datetime.now()
date_string = now.strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
df = pd.read_json("/content/drive/MyDrive/003/dataok.json")

In [ ]:
# df = df.iloc[:200,:]
# df = pd.concat([df,df,df,df]).reset_index(drop=True)

In [ ]:
df

In [ ]:
!pip install sentencepiece

In [ ]:
rutabase= '/content/drive/MyDrive/003'

In [ ]:
def GenerarDirectorio(name):
    directorio = os.path.join(rutabase, name)
    if not os.path.exists(directorio):
        os.makedirs(directorio)
    return directorio

In [ ]:
if MODEL_TYPE=='mdeberta':
  dataTrainer = pd.read_json(f"{rutabase}/mdebertaTokenizer.json")
  dataEvaluation = pd.read_json(f"{rutabase}/mdebertaTokenizereval.json")
  datatest = pd.read_json(f"{rutabase}/mdebertaTokenizertest.json")
elif MODEL_TYPE=='deberta':
  dataTrainer = pd.read_json(f"{rutabase}/ebertaTokenizer.json")
  dataEvaluation = pd.read_json(f"{rutabase}/ebertaTokenizereval.json")
  datatest = pd.read_json(f"{rutabase}/mdebertaTokenizertest.json")


In [ ]:
datatest=datatest.iloc[:50,:]

In [ ]:
datatest

In [ ]:
dataTrainer=dataTrainer.iloc[:250,:]

In [ ]:
dataTrainer

In [ ]:
dataEvaluation=dataEvaluation.iloc[:120,:]

In [ ]:
dataEvaluation

In [ ]:
class MyDataset(Dataset):             # define una nueva clase MyDataset que hereda de Dataset
    def __init__(self, dataframe):    # define el constructor  "__init__"  que toma un solo argumento dataframe
        #print(dataframe)
        self.len = len(dataframe)   # calcula la longitud de la entrada dataframe usando la funcion "len" y la almacena como una variable de instancia "self.len"
        self.data = dataframe       # se asigna la entrada dataframe a una variable de instancia "self.data"


    def __getitem__(self, index):   # define el método "__getitem__" que toma un solo argumento index
        ''' el metodo __getitem__ devuelve un diccionario que contiene cuatro claves: 'input_ids', 'attention_mask', 'labels'y 'added_features' '''

        input_ids = torch.tensor(self.data.text_vec.iloc[index]).cpu() # almacena las características de los datos de "text_vec" ​​que se han convertido en un vector de longitud fija.
        #attention_mask = torch.ones([input_ids.size(0)]).cpu()  # attention_mask almacena los elementos de entrada que se debe prestar atención y cuáles se deben ignorar
        #
        mask = torch.ones(input_ids.shape,dtype=int)#Crear un tensor con el mismo tamaño que input_ids lleno de unos:
        pad_positions = (input_ids == 0)#Identificar las posiciones en input_ids que contienen el token especial [PAD]
        mask[pad_positions] = 0 #Actualizar las posiciones correspondientes en mask a cero:
        attention_mask = mask #Actualizar las posiciones correspondientes en mask a cero:
        #
        label = self.data.same.iloc[index] # almacena un valor escalar que representa la etiqueta de salida para la puntuación de complejidad
        targets = torch.tensor([1 - label, label])  #ojo probar ESTO ES NUEVO
        return {
            'input_ids': input_ids,               # devuelve las características de entrada para el punto de datos
            'attention_mask': attention_mask,     # devuelve la máscara de atención para el punto de datos
            'labels': targets                    # devuelve un valor escalar que representa la puntuación de complejidad
        }

    def __len__(self):
        return self.len   # devuelve la longitud del conjunto de datos personalizado

In [ ]:
train_set, eval_dataset = MyDataset(dataTrainer), MyDataset(dataEvaluation)

In [ ]:
train_set.len, eval_dataset.len

In [ ]:
train_set

In [ ]:
def c_at_1(train_data, test_data, threshold=0.5):
      n = float(len(test_data))
      nc, nu = 0.0, 0.0

      for gt_score, pred_score in zip(train_data, test_data):
        if pred_score == 0.5:
          nu += 1
        elif (pred_score > 0.5) == (gt_score > 0.5):
          nc += 1.0

      return (1 / n) * (nc + (nu * nc / n))

def binarize(y, threshold=0.5, triple_valued=False):
    y = np.array(y)
    y = np.ma.fix_invalid(y, fill_value=threshold)
    if triple_valued:
        y[y > threshold] = 1
    else:
        y[y >= threshold] = 1
    y[y < threshold] = 0
    return y

def f_05_u_score(train_data, test_data, pos_label=1, threshold=0.5):

      test_data = binarize(test_data)

      n_tp = 0
      n_fn = 0
      n_fp = 0
      n_u = 0

      for i, pred in enumerate(test_data):
        if pred == threshold:
          n_u += 1
        elif pred == pos_label and pred == train_data[i]:
          n_tp += 1
        elif pred == pos_label and pred != train_data[i]:
          n_fp += 1
        elif train_data[i] == pos_label and pred != train_data[i]:
          n_fn += 1

      return (1.25 * n_tp) / (1.25 * n_tp + 0.25 * (n_fn + n_u) + n_fp)

def brier_score(train_data, test_data):
      try:
        return 1 - brier_score_loss(train_data, test_data)
      except ValueError:
        return 0.0

def auc_score(train_data, test_data):
    try:
        return roc_auc_score(train_data, test_data)
    except ValueError:
        return 0.0


In [ ]:
def compute_metrics(p: EvalPrediction): # calcula diversas métricas de evaluación
  preds = np.argmax(p.predictions,axis=-1)
  preds = np.squeeze(preds)
  labels = np.argmax(p.label_ids,axis=-1)
  labels = np.squeeze(labels)
  precision = precision_score(labels, preds, average='micro')
  precision_macro = precision_score(labels, preds, average='macro')
  auc = auc_score(labels, preds)
  #mse = metrics.mean_squared_error(labels, preds)# Se toma el primer indice del label
  #print("SIZES::::",labels.shape, preds.shape)


  return {
      'Accuracy-RC': accuracy_score(labels, preds),
          'auc': auc,
          'c@1': c_at_1(labels, preds),
          'f_05_u': f_05_u_score(labels, preds),
          'F1': f1_score(labels, preds, average = 'micro'),
          'brier': brier_score(labels, preds),
          'precision micro': precision,
          'precision macro': precision_macro
          }

In [ ]:
# Definir bien los argumentos
args = TrainingArguments(
    output_dir='/content/drive/MyDrive/003/output',  # Ruta del directorio de salida donde se guardarán los resultados del entrenamiento
    evaluation_strategy='epoch',  # Evaluación del modelo al final de cada época
    num_train_epochs=7,  # Número total de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del lote de entrenamiento por dispositivo. Ajustar según la memoria GPU disponible
    per_device_eval_batch_size=16,  # Tamaño del lote de evaluación por dispositivo. Ajustar según la memoria GPU disponible
    learning_rate=5e-5,  # Tasa de aprendizaje utilizada en el entrenamiento
    overwrite_output_dir=True,  # Sobrescribir el directorio de salida si ya existe
    remove_unused_columns=False,  # No eliminar columnas no utilizadas del conjunto de datos
    logging_dir='/content/drive/MyDrive/003/logs',  # Ruta del directorio donde se guardarán los archivos de registro del entrenamiento
    logging_steps=10,  # Número de pasos después de los cuales se realizará el registro
    save_strategy='epoch',  # Estrategia de guardado del modelo: al final de cada época
    save_total_limit=10,  # Límite total de modelos guardados
    load_best_model_at_end=True,  # Cargar el mejor modelo al final del entrenamiento
    warmup_steps=10,  # Número de pasos de calentamiento antes de ajustar la tasa de aprendizaje
    weight_decay=0.03,  # Factor de decaimiento de peso para la regularización L2
    adam_epsilon=1e-8,  # Epsilon para el optimizador Adam, utilizado para la estabilidad numérica
    adam_beta1=0.5,  # Coeficiente beta1 para el optimizador Adam
    adam_beta2=0.5,  # Coeficiente beta2 para el optimizador Adam
    lr_scheduler_type='cosine',  # Tipo de programador de tasa de aprendizaje: programador coseno curva de aprendizaje  entre el eje x
    gradient_accumulation_steps=1,  # Número de pasos de acumulación de gradiente antes de realizar una actualización de parámetros
    max_grad_norm=5.0,  # Valor máximo de la norma del gradiente para evitar explosiones de gradiente
    save_steps=10  # Número de pasos después de los cuales se guarda el modelo
    )


In [ ]:
# Metricas realizadas durante la predicción
def metricas_preds(preds, label):
  return {
          'F1-macro-RC': f1_score(label, preds, average='macro'),
          'F1-bynary-RC': f1_score(label, preds, average='binary'),
          'Accuracy-RC': accuracy_score(label, preds),
          'Precision_Score_Macro_RC': precision_score(label, preds, average='macro'),
          'Precision_Score_Micro_RC': precision_score(label, preds, average='micro'),
          'Precision_Score_Weighted_RC': precision_score(label, preds, average='weighted')

          }


In [ ]:

class MyTrainer(Trainer):
      def _init_(self, **kwargs):
            super()._init_(**kwargs)


In [ ]:
PATH_historial_optuna = rutabase +"EN-historial_optuna-rango.csv"
PATH_modelo = rutabase + 'EN-ModeloEntrenadoOptuna-rango.pt'
# Guardar los mejores parametros en un archivo
PATH_parametros = rutabase + "EN-mejores_hiperparametros-rango.json"
# Guardar graficas y resultados
PATH_resultados_preds = rutabase +"EN-resultado_metricas_preds-rango.json"
PATH_predicciones = rutabase +"EN-dataPreds_predicciones-rango.json"
PATH_imagen_matriz = rutabase +"EN-matriz_confusion_preds-rango.png"
PATH_grafico_optuna = rutabase +"EN-grafico_optuna-rango.png"
PATH_grafico_optuna_param = rutabase +"EN-grafico_optuna_trials-rango.png"
PATH_historial_optuna = rutabase +"EN-historial_optuna-rango.csv"
PATH_result_train = rutabase +"EN-resultados-train-metricas.json"
PATH_result_eval = rutabase +"EN-resultados-eval-metricas.json"

In [ ]:
class StackedCLSModel(nn.Module):
      def __init__(self,lstm_dict, model=MODEL, model_type=MODEL_TYPE):
        super(StackedCLSModel, self).__init__()
                            # método __init__ nos aseguramos de que el módulo de red neuronal herede las propiedades y métodos de la clase nn.Module
        self.model = MODEL # almacene la ruta al modelo pre-entrenado el tipo de modelo
        self.model_type = MODEL_TYPE # almacene la ruta al tipo de modelo
        self.Fusion = nn.Parameter(torch.zeros(12,1)) # se utiliza para crear un tensor de tamaño (12, 1) lleno de ceros y convertirlo en un parámetro de la red neuronal para que pueda ser optimizado durante el entrenamiento.
        self.dropout =  nn.Dropout(lstm_dict['dropout_rate']) # crea una capa de abandono con una probabilidad de abandono de 0,3

        #escoger la función de activación mas apropiada.
        if lstm_dict['func_activation'] == "tanh":
          self.funActivacion = nn.Tanh()
        elif lstm_dict['func_activation'] == "relu":
          self.funActivacion = nn.ReLU()
        elif lstm_dict['func_activation'] == "gelu":
          self.funActivacion = nn.GELU()

        self.lin1 = nn.Linear(768, 384)#crea una capa lineal en PyTorch que se utiliza en una red neuronal
                                       #para realizar una transformación lineal de las características de entrada de tamaño 768 a características de salida de tamaño 128
                                       #que se utiliza en un modelo BERT para procesar la entrada de texto.
        self.lin2 = nn.Linear(384, 2)
        self.loss_func = nn.CrossEntropyLoss() # establece la función de pérdida que se utilizará durante el entrenamiento
      def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        if self.model_type == "bert":
          cls_tensors = torch.stack([outputs[2][n][:, 0, :] for n in range(1, 13)])
        elif self.model_type == "deberta":
          cls_tensors = torch.stack([outputs[1][n][:, 0, :] for n in range(1, 13)])
        t_cls_tensors = cls_tensors.transpose(1, 0)
        t_cls_tensors_mean = torch.mean(t_cls_tensors, dim=1)  # Reducción de la dimensión 12 a 2
        x = self.lin1(t_cls_tensors_mean)
        x = self.dropout(x)
        x = self.funActivacion(x)
        logit = self.lin2(x)
        loss = None
        if labels is not None:
          loss = self.loss_func(logit, labels.float())

        return SequenceClassifierOutput(loss=loss, logits=logit)
      def predict(self, input_ids, attention_mask,labels):
          logits = self.forward(input_ids, attention_mask, labels=None)
          # predicciones = logits.logits.argmax(dim=1)
          # predicciones =np.argmax(predicciones.tolist(),axis=-1)
          # return predicciones.tolist()
          logits = self.forward(input_ids, attention_mask, labels=None)
          logits = logits.logits
          logits = logits.detach().numpy() #lo convertimos en un tipo de dato numpy para poder manipularlo
          logits = np.argmax(logits,axis=-1) #aplicamos la función argmax para obtener el indice con mayor valor
          logits = logits[0]
          return logits

In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Entrenamiento Optuna y objetivo
#----------------------------------------------------------------------------------------------------

###### Definición de Hiperparámetros ######
#funcion de activación
activation_options = ["tanh","relu", "gelu"]

#Dropout
dropout_min = 0.2
dropout_max = 0.5

#Learning rate
lr_rate_min = 3e-5
lr_rate_max = 5e-5

#epochs
MIN_EPOCHS = 1
MAX_EPOCHS = 5

#Batchs
BATCHS_options = [8,16,32]


def objective(trial: optuna.Trial):

    lstm_dict = {'dropout_rate': trial.suggest_loguniform("dropout", low= dropout_min, high= dropout_max),
                 'func_activation': trial.suggest_categorical("func_activ", activation_options)}

    model = StackedCLSModel(lstm_dict,MODEL,MODEL_TYPE) #inicialización del modelo
    model.to(device)

    training_args = TrainingArguments(
        output_dir='output', #Directorio de salida donde se guardarán los archivos generados durante el entrenamiento
        #evaluation_strategy='epoch', #la evaluación se realiza después de cada época
        learning_rate=trial.suggest_loguniform('learning_rate', low=lr_rate_min, high=lr_rate_max),
        #num_train_epochs=3,
        num_train_epochs=trial.suggest_int('num_epochs', low = MIN_EPOCHS,high = MAX_EPOCHS),
        remove_unused_columns=False, #se eliminarán las columnas no utilizadas en los datos de entrenamiento y evaluación
        per_device_train_batch_size=trial.suggest_categorical("batch_opt", BATCHS_options),
        per_device_eval_batch_size=trial.suggest_categorical("batch_opt", BATCHS_options))

    trainer = MyTrainer(
        model=model,
        compute_metrics=compute_metrics,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=eval_dataset)

    trainer.train()


    #Se define cual sera el objetivo a minimizar o maximizar
    evaluation_result = trainer.evaluate()
    #validation_loss = evaluation_result['eval_loss']
    #train_loss = result.training_loss
    accuracy = evaluation_result['eval_Accuracy-RC']
    f1_macro = evaluation_result['eval_F1']

    #return validation_loss, train_loss , accuracy
    return f1_macro , accuracy


In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Definición de early stop - Optuna
#----------------------------------------------------------------------------------------------------

OPTUNA_EARLY_STOPING = 4 # Aqui se define el stop, si los resultados de optuna siguen siendo iguales luego de 10 trials seguidos, entonces detener la optimización

class EarlyStoppingExceeded(optuna.exceptions.OptunaError):
    early_stop = OPTUNA_EARLY_STOPING
    early_stop_count = 0
    best_score = None

def early_stopping_opt(study, trial):
    resultados_optuna = max(study.best_trials, key=lambda t: t.values[1])
    best_f1 = resultados_optuna.values[0]

    if EarlyStoppingExceeded.best_score == None:
      EarlyStoppingExceeded.best_score = best_f1

    if best_f1 < EarlyStoppingExceeded.best_score:
        EarlyStoppingExceeded.best_score = best_f1
        EarlyStoppingExceeded.early_stop_count = 0
    else:
      if EarlyStoppingExceeded.early_stop_count > EarlyStoppingExceeded.early_stop:
            EarlyStoppingExceeded.early_stop_count = 0
            best_score = None
            raise EarlyStoppingExceeded()
      else:
            EarlyStoppingExceeded.early_stop_count=EarlyStoppingExceeded.early_stop_count+1
    #print(f'EarlyStop counter: {EarlyStoppingExceeded.early_stop_count}, Best score: {study.best_value} and {EarlyStoppingExceeded.best_score}')
    return


In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Creación del estudio Optuna
#----------------------------------------------------------------------------------------------------

print('Activación del estudio de Optuna')
study = optuna.create_study(directions=["maximize", "maximize"]) # multiples objetivos

#para ejecución sin early stop
#study.optimize(func=objective, n_trials=NUM_TRIALS)

# ejecucion con early stop
try:
    study.optimize(objective, timeout=60, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No hay nuevos mejores puntajes en iteraciones {OPTUNA_EARLY_STOPING}')

#### Guardar historial ###
trials_df = study.trials_dataframe()
trials_df.to_csv(PATH_historial_optuna, index=False)



In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Gráficas
#----------------------------------------------------------------------------------------------------

### Grafico 1 ###
grafico_optuna1 = optuna.visualization.matplotlib.plot_pareto_front(study, target_names=["f1", "accuracy"])
# Ajustar el tamaño de la figura
fig1 = grafico_optuna1.figure
fig1.set_size_inches(20, 10)  # Ajusta el tamaño según tus necesidades
fig1.savefig(PATH_grafico_optuna, dpi=400)  # Guardado de imagen


### Grafico 2 ###
grafico_optuna2 = optuna.visualization.matplotlib.plot_optimization_history(study, target=lambda t: t.values[0])
# Ajustar el tamaño de la figura
fig2 = grafico_optuna2.figure
fig2.set_size_inches(20, 10)  # Ajusta el tamaño según tus necesidades
fig2.savefig(PATH_grafico_optuna_param, dpi=400)  # Guardado de imagen


In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Parametros encontrados con Optuna
#----------------------------------------------------------------------------------------------------

print(study.best_trials)
resultados_optuna = max(study.best_trials, key=lambda t: t.values[1])
print(resultados_optuna.values)

print('Encontrar los mejores parámetros del estudio')

best_dropout = float(resultados_optuna.params['dropout'])
best_func_act = resultados_optuna.params['func_activ']
best_lr = float(resultados_optuna.params['learning_rate'])
best_epochs = float(resultados_optuna.params['num_epochs'])
best_batch = resultados_optuna.params['batch_opt']

print('Extraer los mejores parámetros de estudio')

print(f'El mejor dropout es: {best_dropout}')
print(f'La mejor funcion de activación es: {best_func_act}')
print(f'El mejor learning rate is: {best_lr}')
print(f'El mejor epochs is: {best_epochs}')
print(f'El mejor batch is: {best_batch}')


print('Crear diccionario de los mejores hiperparámetros')
best_hp_dict = {
    'best_dropout' : best_dropout,
    'best_func_act' : best_func_act,
    'best_learning_rate' : best_lr,
    'best_epochs': best_epochs,
    'best_batch' : best_batch
}



In [ ]:

# Abrir el archivo en modo escritura
with open(PATH_parametros, "w") as archivo:
    # Guardar el diccionario como JSON en el archivo
    json.dump(best_hp_dict, archivo)


In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Entrenamiento con los mejores parametros
#----------------------------------------------------------------------------------------------------

lstm_dict = None
lstm_dict = {'dropout_rate': best_dropout,
              'func_activation': best_func_act}

#model = None
model = StackedCLSModel(lstm_dict,MODEL, MODEL_TYPE) #inicialización del modelo

# definir bien los arg
training_args = TrainingArguments(  # configuracion o argumentos de la forma en que se realizará el entrenamiento y evaluacion del modelo
    output_dir='output', #Directorio de salida donde se guardarán los archivos generados durante el entrenamiento
    #evaluation_strategy='epoch', #la evaluación se realiza después de cada época
    learning_rate=best_lr,
    #eval_steps=1000, Número de pasos de actualización entre dos evaluaciones si
    #num_train_epochs=3, #número de épocas de entrenamiento que se realizarán
    num_train_epochs = best_epochs,
    remove_unused_columns=False, #se eliminarán las columnas no utilizadas en los datos de entrenamiento y evaluación
    per_device_train_batch_size=best_batch, #Tamaño del lote de entrenamiento por dispositivo
    per_device_eval_batch_size=best_batch, #Tamaño del lote de evaluación por dispositivo
)


In [ ]:
trainer = MyTrainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=eval_dataset)


In [ ]:

print("Training")
result = trainer.train()
evaluation_result = trainer.evaluate()

with open(PATH_result_train, "w") as archivo:
    # Guardar el diccionario como JSON en el archivo
    json.dump(result, archivo)

with open(PATH_result_eval, "w") as archivo:
    # Guardar el diccionario como JSON en el archivo
    json.dump(evaluation_result, archivo)


In [ ]:

#----------------------------------------------------------------------------------------------------
#                    Guardado del modelo
#----------------------------------------------------------------------------------------------------

torch.save(trainer.model.state_dict(), PATH_modelo)
print("FINALIZADO ---> modelo guardado")

In [ ]:
#----------------------------------------------------------------------------------------------------
#                    Carga del modelo
#----------------------------------------------------------------------------------------------------

#carga del modelo base
#modelocarga = StackedCLSModel(lstm_dict,MODEL, MODEL_TYPE)

#carga del modelo entrenado al modelo base
#modelocarga.load_state_dict(torch.load(PATH_modelo, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
#modelocarga.eval()
model.to(device)
model.eval()

In [ ]:
datatest

In [ ]:
from torch.utils.data import DataLoader

# Crear el dataset
mydata = MyDataset(datatest)

# Ajustar el tamaño del lote según tus necesidades
batch_size = 16

# Crear un DataLoader para recorrer el dataset en lotes
dataloader = DataLoader(mydata, batch_size=batch_size, shuffle=True)

# Recorrer el DataLoader por lotes
for batch in dataloader:
    # Obtener los datos del lote
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    # Mover los tensores al dispositivo adecuado (por ejemplo, GPU si está disponible)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    # Aquí deberías cargar el modelo previamente entrenado o definir uno nuevo si es necesario

    # Llamar al método predict en el modelo
    predicciones = model.predict(input_ids, attention_mask)

    # Aquí puedes realizar cualquier operación adicional que necesites con las predicciones

    # Por ejemplo, puedes crear un DataFrame con las predicciones y guardar los resultados
    data = {'changes': predicciones}
    texts = pd.DataFrame(data)
    texts.to_json(os.path.join(folderComplete, f'solution-problem-{instansc}.json'), orient='records')


In [ ]:
torch.cuda.empty_cache()


In [ ]:
model

In [ ]:
datatest['same'] = datatest['label'].apply(lambda x: 1 if x == 'human' else 0) #crea en el dataframe una columna con el nombre de "same" y compara con la columna "label" si es human -> 1 caso contrario ->0
#vectorización
datatest['text_vec'] = datatest.apply(lambda r: vectorize_text(r['text'], 512), axis=1)
#llamado del dataset
df_preds_set = MyDataset(df_preds)

df_preds["predict"] = ""

predict_test = []
data_test =[]

for i in range(len(df_preds)):
  predict_test = model.predictOJO(torch.stack([df_preds_set[i]['input_ids']]),torch.stack([df_preds_set[i]['attention_mask']]))

  data_test.append(predict_test) #agregamos los datos en un array que continene el orden de las filas que vamos a predecir
df_preds['predict'] = data_test # agregamos ese resultado en una columna del dato que hicimos la predicción


In [ ]:
study.best_trials

In [ ]:
trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
study.best_trial.params

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna_visualization.plot_slice(study)

In [ ]:
trial = study.best_trial


In [ ]:
trial.params

In [ ]:
optuna.visualization.plot_contour(study, params=['learning_rate', 'num_layers'])

In [ ]:
trial = study.best_trials

In [ ]:
trial

In [ ]:
study.optimize(objective, n_trials=7)  # Ajusta el número de ensayos según tus necesidades


In [ ]:
result=None
result=trainer.train()

In [ ]:
result.metrics

In [ ]:


def group_by_property(metrics):
    properties = set()
    for metric in metrics:
        for property in metric:
            properties.add(property)
    grouped_metrics = {}
    for property in properties:
        grouped_metrics[property] = [metric for metric in metrics if property in metric]
    return grouped_metrics
def agrupar_propiedades(lista):
    resultado = {}
    for diccionario in lista:
        for propiedad, valor in diccionario.items():
            # Reemplazar espacios y caracteres especiales en la propiedad con guiones bajos
            propiedad = propiedad.replace(' ', '_').replace('-', '_')
            if propiedad not in resultado:
                resultado[propiedad] = []
            resultado[propiedad].append(valor)
    return resultado

def _getvalores(data,key):{
  [metric[key] for metric in data if key in data]
}



def normalizar_propiedades(lista):
    for diccionario in lista:
        for propiedad in list(diccionario.keys()):
            propiedad_normalizada = propiedad.replace(" ", "_").replace("-", "_").replace(".", "_").replace(":", "_").replace("/", "_")
            if propiedad != propiedad_normalizada:
                diccionario[propiedad_normalizada] = diccionario.pop(propiedad)
    return lista


In [ ]:
def generarGrafico(metrics,numero=2):
    metricas2 = agrupar_propiedades(metrics)
    fecha_hora = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    # Configurar el gráfico
    plt.figure(figsize=(10, 8))
    plt.title('Métricas de entrenamiento por época')
    plt.xlabel('Época')
    plt.ylabel('Valor de la métrica')
    plt.ylim(0, 1.5)

    # Graficar cada métrica con longitud distinta a 7
    for name, valores in metricas2.items():
        if len(valores) == numero:  # Verificar si la longitud es igual a 7
            plt.plot(range(len(valores)), valores, label=name,marker='o')

    # Agregar leyenda y mostrar gráfico
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)

    # Establecer valores y etiquetas en el eje y con todos los decimales
    plt.yticks([i / 10 for i in range(15)])

    # Agregar fecha y hora al pie de la gráfica
    plt.figtext(0.99, 0.01, fecha_hora, ha='right', va='bottom')
    ruta =GenerarDirectorio('Graficos')
       # Ajustar el tamaño de la gráfica dentro de la ventana
     # Guardar el gráfico en la ubicación especificada con mayor resolución
    plt.tight_layout()

    plt.savefig(os.path.join(ruta, f'{date_string}_grafico.png'), dpi=1800,bbox_inches='tight')

    plt.show()


In [ ]:
def GenerarMatrizConfuncion(matriz_confusion):
    # Configurar el gráfico
    plt.imshow(matriz_confusion, cmap='Blues', interpolation='nearest')
    plt.title('Matriz de Confusión')
    plt.colorbar()

    # Etiquetas de los ejes x e y
    tick_marks = np.arange(len(matriz_confusion))
    plt.xticks(tick_marks, ['Negativo', 'Positivo'])
    plt.yticks(tick_marks, ['Negativo', 'Positivo'])

    # Mostrar los valores de la matriz de confusión en cada celda
    thresh = matriz_confusion.max() / 2.
    for i, j in np.ndindex(matriz_confusion.shape):
        plt.text(j, i, format(matriz_confusion[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if matriz_confusion[i, j] > thresh else "black")

    plt.figtext(0.99, 0.01, date_string, ha='right', va='bottom')
    ruta =GenerarDirectorio('Graficos')
    # Ajustar los márgenes del gráfico
    plt.tight_layout()

    plt.savefig(os.path.join(ruta, f'{date_string}_matrizConfusion.png'), dpi=1800,bbox_inches='tight')
    # Mostrar el gráfico
    plt.show()

In [ ]:
metrics=trainer.state.log_history
metrics=normalizar_propiedades(metrics)


In [ ]:
generarGrafico(metrics,args.num_train_epochs)

In [ ]:
direccion=GenerarDirectorio('best_model')
rutamodel=os.path.join(direccion,'best_model.pth')

In [ ]:
import torch
torch.save(model, rutamodel)

In [ ]:
import torch
modelo_cargado = torch.load(rutamodel)

In [ ]:
modelo_cargado

In [ ]:
train_prueba=train_set

In [ ]:
train_input_ids = [train_prueba[i]['input_ids'] for i in range(len(train_prueba))]
train_attention_mask = [train_prueba[i]['attention_mask'] for i in range(len(train_prueba))]


In [ ]:
train_input_ids_tensor = torch.stack(train_input_ids)  # Convertir la lista en un tensor
train_attention_mask_tensor = torch.stack(train_attention_mask)  # Convertir la lista en un tensor

In [ ]:

# Realizar las predicciones con el modelo cargado
predicciones = modelo_cargado.predict(train_input_ids_tensor, train_attention_mask_tensor)


In [ ]:
predicciones

In [ ]:
prediciones = trainer.predict(train_set)

In [ ]:
prediciones

In [ ]:
predicciones=prediciones

In [ ]:
reponsestrue= predicciones.predictions

In [ ]:
reponsestrue

In [ ]:
preds = np.argmax(reponsestrue,axis=-1)

In [ ]:
preds

In [ ]:
labels_idsbin= torch.from_numpy(predicciones.label_ids)

In [ ]:
labels_idsbin

In [ ]:
bin_labels = torch.argmax(labels_idsbin, dim=1)

In [ ]:
bin_labels

In [ ]:
matriz_confusion = confusion_matrix(bin_labels, preds)

In [ ]:
matriz_confusion

In [ ]:
GenerarMatrizConfuncion(matriz_confusion)

In [ ]:
evaluacion = trainer.evaluate()

In [ ]:
evaluacion

In [ ]:
print (prediciones.predictions.shape, prediciones.label_ids.shape)
print (prediciones.predictions, prediciones.label_ids)

In [ ]:
evaluate

In [ ]:
evaluacio=model.eval()

In [ ]:
torch.cuda.empty_cache()